In [8]:
import numpy as np
import pandas as pd
import csv
from sklearn.utils import shuffle


In [50]:
# Import player data and historical results for all countries
raw_data_path = 'data/'

prior = pd.read_csv(raw_data_path + 'results.csv')
fifa_dataset = pd.read_csv(raw_data_path + 'CompleteDataset.csv')

# drop unwanted fields from both data sets
fifa_dataset.drop(labels = ['Unnamed: 0', 'Age', 'Photo', 'Flag', 'Club', 'Club Logo', 'Wage'], axis = 1, inplace = True)
prior.drop(labels = ['city', 'country', 'tournament'], axis = 1, inplace = True)
prior = prior[prior.date > '1990-01-01']

pdata = pd.read_csv('data/WC18_processed.csv')
prdata = pd.read_csv('data/WC18_prior.csv')



# only interested in countries that are qualified for the WC
# maybe reduce this to 16 countries post group stage for faster computation (other 2^32 simulations)
qualified_countries = ['Egypt', 'Morocco', 'Nigeria', 'Senegal', 'Tunisia', 'Australia', 'Iran', 'Japan', 'Korea Republic', 'Saudi Arabia', 'Belgium', 'Croatia', 'Denmark', 'England', 'France', 'Germany', 'Iceland', 'Poland', 'Portugal', 'Russia', 'Serbia', 'Spain', 'Sweden', 'Switzerland', 'Costa Rica', 'Mexico', 'Panama', 'Argentina', 'Brazil', 'Colombia', 'Peru', 'Uruguay']

In [51]:
# get data for qualified countries only
fifa_dataset = fifa_dataset[fifa_dataset['Nationality'].isin(qualified_countries)].fillna(0)
prior = prior[(prior['home_team'].isin(qualified_countries) & prior['away_team'].isin(qualified_countries))]

# get remaining potential
fifa_dataset['Remaining Potential'] = fifa_dataset['Potential'] - fifa_dataset['Overall']

# get only one preferred position (first only)
# i wrote a function to get the position with max value - it's in archives -- but this is simpler
fifa_dataset['Preferred Position'] = fifa_dataset['Preferred Positions'].str.split().str[0]

In [11]:
# set the general formations for squads to get the max potential 
squad_352_strict = ['GK', 'LB|LWB', 'CB', 'RB|RWB', 'LM|W$', 'RM|W$', 'CM', 'CM|CAM|CDM', 'CM|CAM|CDM', 'W$|T$', 'W$|T$']
squad_442_strict = ['GK', 'LB|LWB', 'CB', 'CB', 'RB|RWB', 'LM|W$', 'RM|W$', 'CM', 'CM|CAM|CDM', 'W$|T$', 'W$|T$']
squad_433_strict = ['GK', 'LB|LWB', 'CB', 'CB', 'RB|RWB', 'CM|LM|W$', 'CM|RM|W$', 'CM|CAM|CDM', 'W$|T$', 'W$|T$', 'W$|T$']
squad_343_strict = ['GK', 'LB|LWB', 'CB', 'RB|RWB', 'LM|W$', 'RM|W$', 'CM|CAM|CDM', 'CM|CAM|CDM', 'W$|T$', 'W$|T$', 'W$|T$']
squad_532_strict = ['GK', 'LB|LWB', 'CB|LWB|RWB', 'CB|LWB|RWB', 'CB|LWB|RWB', 'RB|RWB', 'M$|W$', 'M$|W$', 'M$|W$', 'W$|T$', 'W$|T$']

squad_352_adj = ['GK', 'B$', 'B$', 'B$', 'M$|W$|T$', 'M$|W$|T$', 'M$|W$|T$', 'M$|W$|T$', 'M$|W$|T$', 'W$|T$|M$', 'W$|T$|M$']
squad_442_adj = ['GK', 'B$', 'B$', 'B$', 'B$', 'M$|W$|T$', 'M$|W$|T$', 'M$|W$|T$', 'M$|W$|T$', 'W$|T$|M$', 'W$|T$|M$']
squad_433_adj = ['GK', 'B$', 'B$', 'B$', 'B$', 'M$|W$|T$', 'M$|W$|T$', 'M$|W$|T$', 'W$|T$|M$', 'W$|T$|M$', 'W$|T$|M$']
squad_343_adj = ['GK', 'B$', 'B$', 'B$', 'M$|W$|T$', 'M$|W$|T$', 'M$|W$|T$', 'M$|W$|T$', 'W$|T$|M$', 'W$|T$|M$', 'W$|T$|M$']
squad_532_adj = ['GK', 'B$', 'B$', 'B$', 'B$', 'B$', 'M$|W$|T$', 'M$|W$|T$', 'M$|W$|T$', 'W$|T$|M$', 'W$|T$|M$']

midfield = ['CDM', 'CM', 'RM', 'LM']
defense = ['GK', 'LB', 'CB', 'RB', 'LWB', 'RWB']
attack = ['LW', 'RW', 'ST', 'CAM']

squad_list_strict = [squad_352_strict, squad_442_strict, squad_433_strict, squad_343_strict, squad_532_strict]
squad_list_adj = [squad_352_adj, squad_442_adj, squad_433_adj, squad_343_adj, squad_532_adj]
squad_name = ['3-5-2', '4-4-2', '4-3-3', '3-4-3', '5-3-2']

In [12]:
def get_country_roster(country):
# get all players in a country
    return fifa_dataset[fifa_dataset['Nationality'] == country]

def get_best_squad_n(formation, nationality, df, measurement = 'Potential'):
# finds the best 11 players given formation    
# returns overall_squad_rating, squad_list, squad_stats (i.e. positions and ratings)
    df_copy = df.copy()
    df_copy = df_copy[df_copy['Nationality'] == nationality]
    store = []
    team = []
    for i in formation:
        store.append([df_copy.loc[[df_copy[df_copy['Preferred Position'].str.contains(i)][measurement].idxmax()]]['Preferred Position'].to_string(index = False), df_copy[df_copy['Preferred Position'].str.contains(i)][measurement].max()])
        team.append(df_copy.loc[[df_copy[df_copy['Preferred Position'].str.contains(i)][measurement].idxmax()]]['Name'].to_string(index = False))
        df_copy.drop(df_copy[df_copy['Preferred Position'].str.contains(i)][measurement].idxmax(), inplace = True)
    return np.mean([x[1] for x in store]).round(2), np.array(team), pd.DataFrame(np.array(store).reshape(11,2), columns = ['Position', measurement]).to_string(index = False)

def get_best_formation_all(squad_list, country):
# finds the best formation for the given team 
# returns the best_rating, best_squad_list, best_formation, and squad_stats (i.e. positions and ratings)
    best_rating = 0
    best_squad, best_formation = [], []
    for i, formation in enumerate(squad_list):
        potRating, pot_squad, squad_info = get_best_squad_n(formation, country, fifa_dataset, 'Potential')
        if potRating > best_rating:
            best_rating = potRating
            best_squad = pot_squad
            best_formation = squad_name[i]
    return best_rating, best_squad, best_formation, squad_info
    
def get_team_stats(squad_stats):
# gets the stats for attack, defence and midfield for a team
    midRating, defRating, attRating  = [], [], []
    for player in squad_stats.split('\n'):
        pos, rating = player.split()[0], player.split()[-1]
        if pos in midfield: midRating.append(float(rating))
        if pos in defense: defRating.append(float(rating))
        if pos in attack: attRating.append(float(rating))
    return np.mean(midRating).round(2), np.mean(defRating).round(2), np.mean(attRating).round(2)

In [13]:
# create data-set with team information
def generate_team_stats():
    d = []
    for country in qualified_countries:
            best_rating, best_squad, best_formation, squad_stats = get_best_formation_all(squad_list_adj, country)
            mid_rating, def_rating, att_rating = get_team_stats(squad_stats)
            if best_rating > 84.0: tier = 1
            elif best_rating > 77.0: tier = 2
            else: tier = 3
            d.append({'Country': country, 'Rating': best_rating, 'Squad': best_squad, 'Defence': def_rating, 'Midfield': mid_rating, 'Attack': att_rating, 'Formation': best_formation,'Tier': tier})

    processed_data = pd.DataFrame(d).reindex_axis(['Country', 'Squad', 'Formation', 'Tier', 'Rating', 'Attack', 'Midfield', 'Defence'], axis = 1).set_index('Country')
    processed_data.to_csv('data/WC18_processed.csv')
    return processed_data

# create data-set with historical data of match up for prior information



In [14]:
def get_competition_stats():
    teams = []
    seen = []
    for countryA in qualified_countries:
        for countryB in qualified_countries:
            if (countryA == countryB) | ((countryB, countryA) in seen): continue
            seen.append((countryA, countryB))
            attA = float(pdata[pdata['Country'] == countryA].Attack.get_values())
            defA = float(pdata[pdata['Country'] == countryA].Defence.get_values())
            midA = float(pdata[pdata['Country'] == countryA].Midfield.get_values())
            ratA = float(pdata[pdata['Country'] == countryA].Rating.get_values())
            tierA = float(pdata[pdata['Country'] == countryA].Tier.get_values())
            attB = float(pdata[pdata['Country'] == countryB].Attack.get_values())
            defB = float(pdata[pdata['Country'] == countryB].Defence.get_values())
            midB = float(pdata[pdata['Country'] == countryB].Midfield.get_values())
            ratB = float(pdata[pdata['Country'] == countryB].Rating.get_values())
            tierB = float(pdata[pdata['Country'] == countryB].Tier.get_values())
            winA = float(np.sign((prdata[(prdata['CountryA'] == countryA) & (prdata['CountryB'] == countryB)].meanScoreA - prdata[(prdata['CountryA'] == countryA) & (prdata['CountryB'] == countryB)].meanScoreB)))
            teams.append({'attA': attA, 'defA': defA, 'midA': midA, 'ratA':ratA, 'tierA': tierA, 'attB': attB, 'defB': defB, 'midB': midB, 'ratB':ratB, 'tierB': tierB, 'winA': winA })
    competition_stats = pd.DataFrame(teams)
#    competition_stats.to_csv('data/WC18_competition_stats.csv')
    return competition_stats

In [15]:
c_stats = get_competition_stats()
shuffle(c_stats).head(3)

,attA,attB,defA,defB,midA,midB,ratA,ratB,tierA,tierB,winA
316,85.50,85.67,81.67,83.33,87.67,85.5,84.55,84.45,1.0,1.0,1.0
346,92.33,90.00,89.00,85.33,91.50,89.5,90.73,87.73,1.0,1.0,1.0
9,82.33,90.33,74.17,86.33,83.00,91.0,79.00,88.64,2.0,1.0,1.0


In [23]:

for i, c in enumerate(qualified_countries): print i,c

0 Egypt
1 Morocco
2 Nigeria
3 Senegal
4 Tunisia
5 Australia
6 Iran
7 Japan
8 Korea Republic
9 Saudi Arabia
10 Belgium
11 Croatia
12 Denmark
13 England
14 France
15 Germany
16 Iceland
17 Poland
18 Portugal
19 Russia
20 Serbia
21 Spain
22 Sweden
23 Switzerland
24 Costa Rica
25 Mexico
26 Panama
27 Argentina
28 Brazil
29 Colombia
30 Peru
31 Uruguay


In [98]:
fifa_dataset.shape

(12016, 70)

In [20]:
prior.head()

,date,home_team,away_team,home_score,away_score
15785,1990-01-17,Mexico,Argentina,2,0
15799,1990-01-28,Nigeria,Senegal,1,1
15801,1990-02-02,Colombia,Uruguay,0,2
15803,1990-02-02,Iran,Poland,0,2
15806,1990-02-04,Costa Rica,Uruguay,0,2


In [21]:
def _mapper(ids):
    qualified_countries = ['Egypt', 'Morocco', 'Nigeria', 'Senegal', 'Tunisia', 'Australia', 'Iran', 'Japan', 'Korea Republic', 'Saudi Arabia', 'Belgium', 'Croatia', 'Denmark', 'England', 'France', 'Germany', 'Iceland', 'Poland', 'Portugal', 'Russia', 'Serbia', 'Spain', 'Sweden', 'Switzerland', 'Costa Rica', 'Mexico', 'Panama', 'Argentina', 'Brazil', 'Colombia', 'Peru', 'Uruguay']
    return [qualified_countries[i] for i in ids]
    
    

In [22]:
_mapper([20])

['Serbia']

In [52]:
cc, sq = [], []
all_players = []
with open(raw_data_path + 'WC18_23man.txt', 'r') as raw_list:
    for c_list in raw_list:
        cc.append(c_list.split('\'')[0])
        players = c_list.split(':')[-1].split('\n')[0].split(',')
        sq.append(players)
                                   
        

In [53]:
zip(cc, sq)

[('Russia',
  ['Igor Akinfeev',
   'Vladimir Gabulov',
   'Soslan Dzhanaev',
   'Andrey Lunev',
   ' Vladimir Granat',
   'Sergei Ignashevich',
   'Fedor Kudryashov',
   'Ilya Kutepov',
   'Roman Neustadter',
   'Konstantin Rausch',
   'Andrey Semenov',
   'Igor Smolnikov',
   'Mario Fernandes',
   ' Yuri Gazinskiy',
   'Alexsandr Golovin',
   'Alan Dzagoev',
   'Alexsandr Erokhin',
   'Yuri Zhirkov',
   'Daler Kuzyaev ',
   'Roman Zobnin',
   'Alexsandr Samedov',
   'Anton Miranchuk',
   'Alexsandr Tashaev',
   'Denis Cheryshev',
   ' Artem Dzyuba',
   'Aleksey Miranchuk',
   'Fedor Smolov',
   'Fedor Chalov']),
 ('Saudi Arabia',
  ['Assaf Al-Qarny',
   'Mohammed Al-Owais',
   'Yasser Al-Musailem',
   'Abdullah Al-Mayuf',
   ' Mansoor Al-Harbi',
   'Yasser Al-Shahrani',
   'Mohammed Al-Breik',
   'Saeed Al-Mowalad',
   'Motaz Hawsawi',
   'Osama Hawsawi',
   'Omar Hawsawi',
   'Mohammed Jahfali',
   'Ali Al-Bulaihi',
   ' Abdullah Al-Khaibari',
   'Abdulmalek Al-Khaibri',
   'Abdullah

In [65]:
players_list = []
for players in sq:
    for player in players:
        players_list.append(player)
        
players_list

['Igor Akinfeev',
 'Vladimir Gabulov',
 'Soslan Dzhanaev',
 'Andrey Lunev',
 ' Vladimir Granat',
 'Sergei Ignashevich',
 'Fedor Kudryashov',
 'Ilya Kutepov',
 'Roman Neustadter',
 'Konstantin Rausch',
 'Andrey Semenov',
 'Igor Smolnikov',
 'Mario Fernandes',
 ' Yuri Gazinskiy',
 'Alexsandr Golovin',
 'Alan Dzagoev',
 'Alexsandr Erokhin',
 'Yuri Zhirkov',
 'Daler Kuzyaev ',
 'Roman Zobnin',
 'Alexsandr Samedov',
 'Anton Miranchuk',
 'Alexsandr Tashaev',
 'Denis Cheryshev',
 ' Artem Dzyuba',
 'Aleksey Miranchuk',
 'Fedor Smolov',
 'Fedor Chalov',
 'Assaf Al-Qarny',
 'Mohammed Al-Owais',
 'Yasser Al-Musailem',
 'Abdullah Al-Mayuf',
 ' Mansoor Al-Harbi',
 'Yasser Al-Shahrani',
 'Mohammed Al-Breik',
 'Saeed Al-Mowalad',
 'Motaz Hawsawi',
 'Osama Hawsawi',
 'Omar Hawsawi',
 'Mohammed Jahfali',
 'Ali Al-Bulaihi',
 ' Abdullah Al-Khaibari',
 'Abdulmalek Al-Khaibri',
 'Abdullah Otayf',
 'Taiseer Al-Jassim',
 'Houssain Al-Mogahwi',
 'Salman Al-Faraj',
 'Nawaf Al-Abed',
 'Mohamed Kanno',
 'Hattan 

In [66]:
fifa_dataset = fifa_dataset[fifa_dataset['Name'].isin(players_list)].fillna(0)

In [ ]:
fifa_dataset

,Name,Nationality,Overall,Potential,Value,Special,Acceleration,Aggression,Agility,Balance,...,RCM,RDM,RF,RM,RS,RW,RWB,ST,Remaining Potential,Preferred Position
0,Cristiano Ronaldo,Portugal,94,94,€95.5M,2228,89,63,89,63,...,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0,0,ST
10,Sergio Ramos,Spain,90,90,€52M,2153,75,84,79,60,...,74.0,83.0,70.0,71.0,72.0,69.0,81.0,72.0,0,CB
30,Thiago Silva,Brazil,88,88,€34M,2124,70,77,74,68,...,77.0,83.0,69.0,70.0,67.0,68.0,79.0,67.0,0,CB
39,Marcelo,Brazil,87,87,€38M,2209,77,84,77,82,...,80.0,82.0,79.0,81.0,77.0,80.0,84.0,77.0,0,LB
42,David Silva,Spain,87,87,€44M,1977,72,51,92,89,...,81.0,64.0,81.0,82.0,75.0,82.0,65.0,75.0,0,LM
58,Diego Costa,Spain,86,86,€46M,1984,74,93,58,52,...,71.0,60.0,80.0,76.0,83.0,77.0,60.0,83.0,0,ST
63,Miranda,Brazil,86,86,€25.5M,1890,67,83,61,57,...,67.0,79.0,62.0,63.0,63.0,61.0,75.0,63.0,0,CB
67,Pepe,Portugal,86,86,€13.5M,1839,65,94,63,49,...,63.0,76.0,59.0,59.0,61.0,58.0,72.0,61.0,0,CB
80,Jordi Alba,Spain,85,85,€30.5M,2206,93,75,90,86,...,78.0,80.0,78.0,81.0,75.0,80.0,84.0,75.0,0,LB
94,Bernardo Silva,Portugal,84,91,€43.5M,2012,85,58,88,86,...,79.0,66.0,82.0,83.0,75.0,83.0,68.0,75.0,7,RM
